In [3]:
include("./src/EPIREAD.jl")

Main.EPIREAD

In [4]:
function test(path)
    reader = EPIREAD.Reader(path)

    # Pre-allocate record.
    record = EPIREAD.Record()
    i = 0
    while !eof(reader)
        empty!(record)
        read!(reader, record)
        # (i % 100000 == 0) && (i > 0) && println("Processed $i reads")
        i += 1
    end
    println("Finished processing $(i) reads")

    # Finally, close the reader.
    close(reader)
end

test (generic function with 1 method)

In [5]:
path = "./testing/test_epiread_small.bed"
big_bgz_path = "/Volumes/projects/laird/nathan/projects/julia/epiread/testing/test_epiread.bed.bgz"
part_path = "/Volumes/projects/laird/nathan/projects/julia/epiread/testing/part_epiread.bed"
test(path)

Finished processing 100 reads


In [6]:
# Find any reads in the cache that are too far to the left, or on a different chr than provided
function clean_record_cache!(cache::Vector{EPIREAD.Record}, max_end::Int64, new_chr::String)::Vector{EPIREAD.Record}
    reads_to_analyze = Vector{EPIREAD.Record}()
    for (i, record) in enumerate(cache)
        if (EPIREAD.chromend(record) < max_end) || (EPIREAD.chrom(record) ≠ new_chr)
            push!(reads_to_analyze, popat!(cache, i))
        end
    end
    return reads_to_analyze
end

function get_leftmost_chromend(cache::Vector{EPIREAD.Record})::Int64
    if length(cache) > 0
        return minimum(EPIREAD.chromend.(cache))
    else
        return Int64(0)
    end
end

get_leftmost_chromend (generic function with 1 method)

In [8]:
function analyze_read(read::EPIREAD.Record)
	println("analyzing read $(EPIREAD.name(read))/$(EPIREAD.readnum(read)) ($(EPIREAD.chrom(read)))")
end

allsame(x) = all(y->y==first(x),x)

function analyze_record_pair(read1::EPIREAD.Record, read2::EPIREAD.Record)
	if !allsame(EPIREAD.chrom.((read1, read2)))
		for each in (read1, read2)
			analyze_read(each)
		end
	else 
		println("analyzing read pair $(EPIREAD.name(read1))/$(EPIREAD.readnum(read1)), $(EPIREAD.name(read2))/$(EPIREAD.readnum(read2))")
	end
end


analyze_record_pair (generic function with 1 method)

In [9]:
# Reads must be coordinate-sorted, so read pairs might not be adjacent in the file. 
# Need to set a parameter (insert size) to stop looking for mate pair of a read

#= 
Step through reads, one at a time. When we first see a read, we don't know if we'll find a mate for it. 
So we put it into the 'read cache' for a moment. We then keep reading. For each new read we find, we 
perform two checks:
1. if the chromend of the first (leftmost) read in the cache is more than (max_isize) bp away from the chromstart
of the read we're looking at, we are 'out of pairing range'. We take this opportunity to go through the cache
and purge any reads that are impossible to pair anymore, putting them in the 'analyze' queue. We also push our current read to the cache.
2. if the name of the read we're looking at matches the name of a read in the cache, we pull that read out and analyze it with the current read.
(also should check that chrom stays the same...!)
=# 





In [10]:


function test2(path; max_isize = 1000)

    record_cache = Vector{EPIREAD.Record}()
    reader = EPIREAD.Reader(path)

    # Pre-allocate record.
    record = EPIREAD.Record()
    i = 0
    last_chr = String("")
    while !eof(reader)
    # while i < 1000
        empty!(record)
        read!(reader, record)
        name = EPIREAD.name(record)
        chromstart = EPIREAD.chromstart(record)
        current_chr = EPIREAD.chrom(record)
        
        # Find the furthest-left *end* coordinate of all reads in the cache (if it is too far away from the beginning of this read, we'll never be able to pair to it)
        leftmost_chromend = get_leftmost_chromend(record_cache)
        # Test as above, and also check if we're on a new chr.
        if ( (leftmost_chromend < (chromstart - max_isize)) && (leftmost_chromend > 0) ) || ( (last_chr ≠ current_chr) && (last_chr ≠ "") )
            # If either of the above is true, get all the un-pair-able reads out of the cache and analyze them:
            println("cleaning cache because min_end $leftmost_chromend is more than $max_isize away from $chromstart or because last-seen $(last_chr) is different from current $(EPIREAD.chrom(record)) (read $(name))")
            for each in clean_record_cache!(record_cache, chromstart - max_isize, EPIREAD.chrom(record))
                analyze_read(each)
            end
        end
        
        # Ok, any read still in the cache is fair game to pair with our current read. Let's see if we can find a match:
        cached_names = EPIREAD.name.(record_cache)
        if name in cached_names 
            pair_record = popat!(record_cache, findfirst(isequal(name), cached_names))
            analyze_record_pair(record, pair_record)

        # Ok, we couldn't find a match. Save this read for later (maybe its match just hasn't been read yet!)
        else
            push!(record_cache, copy(record))
        end
        i += 1
        last_chr = current_chr
    end
    println("Finished processing $(i) reads")
    # Finally, close the reader.
    close(reader)
    for each in record_cache
        analyze_read(each)
    end
end


test2 (generic function with 1 method)

In [58]:
test2(path)

analyzing read pair A00426:227:H7KMJDMXY:2:2233:29695:6605/2, A00426:227:H7KMJDMXY:2:2233:29695:6605/1
analyzing read pair A00426:227:H7KMJDMXY:2:2205:19551:25081/1, A00426:227:H7KMJDMXY:2:2205:19551:25081/2
analyzing read pair A00426:227:H7KMJDMXY:2:1331:6994:10034/2, A00426:227:H7KMJDMXY:2:1331:6994:10034/1
analyzing read pair A00426:227:H7KMJDMXY:2:2205:14642:2456/2, A00426:227:H7KMJDMXY:2:2205:14642:2456/1
analyzing read pair A00426:227:H7KMJDMXY:2:1230:12653:24126/1, A00426:227:H7KMJDMXY:2:1230:12653:24126/2
analyzing read pair A00426:227:H7KMJDMXY:2:1437:4255:21261/1, A00426:227:H7KMJDMXY:2:1437:4255:21261/2
analyzing read pair A00426:227:H7KMJDMXY:2:2351:20627:7091/1, A00426:227:H7KMJDMXY:2:2351:20627:7091/2
analyzing read pair A00426:227:H7KMJDMXY:2:2305:21938:14121/1, A00426:227:H7KMJDMXY:2:2305:21938:14121/2
analyzing read pair A00426:227:H7KMJDMXY:2:1309:16649:34272/1, A00426:227:H7KMJDMXY:2:1309:16649:34272/2
analyzing read pair A00426:227:H7KMJDMXY:2:2435:11749:27320/2, A0

In [11]:
reader = EPIREAD.Reader(path)
record = EPIREAD.Record()
read!(reader, record)

Main.EPIREAD.Record:
    chromosome: chr1
         start: 10055
           end: 10203
     name/read: A00426:227:H7KMJDMXY:2:2233:29695:6605/1
        strand: +
CpG RLE string: F3x49ax24Fx12Fx5Fx3cxF2x16Fx10FxFx3FxFx5Fx2F3


In [10]:
import Automa
import Automa.RegExp: @re_str
re = Automa.RegExp

machine = let 
	base = re"[FPxMUOSACGTNacgtnD]"
	rep_num = re"[0-9]+"

	first_rle_unit = re.cat(base, re.opt(rep_num))
	subseq_rle_unit = re.cat(base, re.opt(rep_num))
	rle = re.cat(first_rle_unit, re.rep(subseq_rle_unit))

	# Define actions
	first_rle_unit.actions[:enter] = [:enter_first]
	rep_num.actions[:enter] = [:enter_num]
	subseq_rle_unit.actions[:enter] = [:enter_subseq]


	Automa.compile(rle)
end

const actions = Dict(
	:enter_first => :(char = string(data[p]::Char)),
	:enter_num => :(mark = p; hasnum = true),
	:enter_subseq => quote
		reps = hasnum ? data[mark:p-1]::String : "1"
		hasnum = false
		execute_rle(char, reps, startpos, J, V)
		char = string(data[p]::Char)
	end
)

Dict{Symbol, Expr} with 3 entries:
  :enter_first  => :(char = string(data[p]::Char))
  :enter_num    => quote…
  :enter_subseq => quote…

In [2]:
write("./machine.dot", Automa.machine2dot(machine))


403

In [3]:

context = Automa.CodeGenContext();
Automa.generate_init_code(context, machine)

quote
    #= /Users/Nathan.Spix/.julia/packages/Automa/1KOLQ/src/codegen.jl:106 =#
    p::Int = 1
    #= /Users/Nathan.Spix/.julia/packages/Automa/1KOLQ/src/codegen.jl:107 =#
    p_end::Int = 0
    #= /Users/Nathan.Spix/.julia/packages/Automa/1KOLQ/src/codegen.jl:108 =#
    p_eof::Int = -1
    #= /Users/Nathan.Spix/.julia/packages/Automa/1KOLQ/src/codegen.jl:109 =#
    cs::Int = 1
end

In [4]:
using SparseArrays
# Column access is fast; row access is slow...

#= To construct, we need 3 things:
I = vector of row (read) indexes
J = vector of column (position) indexes
V = vector of values
=#

function execute_rle(base::String, reps::Int8, startpos::Int64, J::Vector{Int64}, V::Vector{String})
	println("Storing base $base $reps times")

	if occursin(base, "FPx") # This is an ignore base
		# do stuff
	elseif occursin(base, "MUOSACGTN") # This is a feature base
		# do stuff
	elseif occursin(base, "acgtn") # this is an insertion base
		# do stuff
	else # This must be a deletion base
		@assert base == "D"
		# do stuff
	end
end

function execute_rle(base::String, reps::String, startpos::Int64, J::Vector{Int64}, V::Vector{String})
	execute_rle(base, parse(Int8, reps))
end


execute_rle (generic function with 2 methods)

In [7]:
@eval function parse_rle(data::String, startpos::Int64)
	J, V = Vector{Int64}(),Vector{String}()
	$(Automa.generate_init_code(context, machine))
	p_end = p_eof = lastindex(data)
	mark, reps, char, hasnum = 0, "0", "", false
	$(Automa.generate_exec_code(context, machine, actions))
	
    iszero(cs) || error("failed to parse rle string on byte ", p)
    return nothing
end;

function parse_rle(data::String, startpos::Int)
	parse_rle(data, Int64(startpos))
end

parse_rle (generic function with 1 method)

In [9]:
@time parse_rle("F3x49ax24Fx12Fx5Fx3cxF2x16Fx10FxFx3FxFx5Fx2F3", 100)

StackOverflowError: StackOverflowError: